![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg)

<center>
<h1><font size="+3">GSFC Python Bootcamp</font></h1>
</center>

---

<CENTER>
<H1 style="color:red">
An Introduction to netCDF4
</H1>
</CENTER>

In [ ]:
from __future__ import print_function

# <font color='red'> Useful References </font>

* <A HREF="http://pyhogs.github.io/intro_netcdf4.html">Create and read netCDF files</A>
* <A HREF="https://unidata.github.io/netcdf4-python/netCDF4/index.html">netCDF4 module</A>

## <font color="red"> What we will cover </font>
* Opening a file
* Dimension
* Variables
* Attributes
* Writing data
* Creating groups
* Reading data

## <font color='red'> What is netCDF4?</font>

* Python interface to the netCDF version 4 library.
* Can read and write files in both the new netCDF 4 and the netCDF 3 format.
* Can create files that are readable by HDF5 utilities.
* Relies on NumPy arrays.

In [ ]:
import numpy as np
from netCDF4 import Dataset

#### <font color='red'> Opening a netCDF File</font>

In [ ]:
ncFileName = 'sample_netcdf.nc4'
modeType   = 'w'
fileFormat = 'NETCDF4'
ncfid = Dataset(ncFileName, mode=modeType, format=fileFormat)

* `modeType`: 'w' (write), 'r+' (read and write with an existing file), 'r' (read), or 'a' (append)
* `fileFormat`: 'NETCDF3_CLASSIC', 'NETCDF3_64BIT_OFFSET', 'NETCDF4_CLASSIC', 'NETCDF4', 'NETCDF3_64BIT_DATA'

#### <font color='red'> Creating Dimensions in a netCDF File</font>

In [ ]:
time = ncfid.createDimension('time', None) # infinite dimension
lev  = ncfid.createDimension('lev', 72)
lat  = ncfid.createDimension('lat', 91)
lon  = ncfid.createDimension('lon', 144)

#### <font color='red'>Create Variables</font>

In [ ]:
times      = ncfid.createVariable('time','f8',('time',))
levels     = ncfid.createVariable('lev','i4',('lev',))
latitudes  = ncfid.createVariable('lat','f4',('lat',))
longitudes = ncfid.createVariable('lon','f4',('lon',))

temp       = ncfid.createVariable('temp','f4', \
                                  ('time','lev','lat','lon',))

#### <font color='red'>Adding Variable Attributes</font>

In [ ]:
latitudes.long_name  = 'latitude'
latitudes.units      = 'degrees north'

longitudes.long_name = 'longitude'
longitudes.units     = 'degrees east'

levels.long_name     = 'vertical levels'
levels.units         = 'hPa'
levels.positive       = 'down'

times.long_name      = 'time'
times.units          = 'hours since 0001-01-01 00:00:00.0'
times.calendar       = 'gregorian'

temp.long_name       = 'temperature'
temp.units           = 'K'
temp.missing_value   = 1.0e15

#### <font color='red'>Adding Global Attributes</font>

In [ ]:
import time
ncfid.description = 'Sample netCDF file'
ncfid.history = 'Created for GSFC on' + time.ctime(time.time())
ncfid.source = 'netCDF4 python tutorial'

#### <font color='red'>Writing Data in the File</font>

In [ ]:
latitudes[:]  =  np.arange(-90,91,2.0)
longitudes[:] =  np.arange(-180,180,2.5)
levels[:]     =  np.arange(0,72,1)

num_records   = 5
temp[0:num_records,:,:,:] = np.random.uniform( size=(num_records,
                                                     levels.size,
                                                     latitudes.size,
                                                     longitudes.size))

#### <font color='red'>Printing Dimension Information</font>

In [ ]:
for dim in ncfid.dimensions.values():
     print(dim, dim.isunlimited())

In [ ]:
for name in ncfid.dimensions.keys():
    dim = ncfid.variables[name]
    print(name, dim.dtype, dim.size)

#### <font color='red'>Printing File Attributes</font>

In [ ]:
for att in ncfid.ncattrs():
    print(att+':', getattr(ncfid,att))

In [ ]:
print(ncfid.__dict__)

#### <font color='red'>Printing Variable Information</font>

In [ ]:
for name in ncfid.variables.keys():
    if (name not in ncfid.dimensions.keys()):
       data = ncfid.variables[name]
       print(name, data.units, data.shape, data.dtype, data.dimensions)

In [ ]:
    def print_ncattr(ncfid, key):
        """
           Prints the NetCDF file attributes for a given key

           Parameters: 
             * ncfid:  netCDF file identifier
             * key:    unicode (a valid netCDF4.Dataset.variables key)
        """
        try:
            print(key, '-->')
            print("\t\ttype:", repr(ncfid.variables[key].dtype))
            for ncattr in ncfid.variables[key].ncattrs():
                print('\t\t%s:' % ncattr,\
                      repr(ncfid.variables[key].getncattr(ncattr)))
        except KeyError:
            print("\t\tWARNING: %s does not contain variable attributes" % key)

In [ ]:
print(print_ncattr.__doc__)

In [ ]:
for name in ncfid.variables.keys():
    print_ncattr(ncfid, name)

#### <font color='red'>Create Groups</font>
* We can organize data in hierarchical groups, which are analogous to directories in a filesystem. 
* Groups serve as containers for variables, dimensions and attributes, as well as other groups.

In [ ]:
fcstgrp  = ncfid.createGroup('forecasts')
fcstgrpm = ncfid.createGroup('forecasts/model')

In [ ]:
print(ncfid.groups)

In [ ]:
def walk_group_tree(top):
    """
       Python generator that is used to walk the directory tree.
    """
    values = top.groups.values()
    yield values
    for value in top.groups.values():
        for children in walk_group_tree(value):
            yield children

In [ ]:
for children in walk_group_tree(ncfid):
    for child in children:
        print(child)

##### Add variable to a group

In [ ]:
tempm = ncfid.createVariable('forecasts/model/temp','f4', \
                                  ('time','lev','lat','lon',))

tempm.long_name       = 'temperature (model)'
tempm.units           = 'K'
tempm.missing_value   = 1.0e15

tempm[0:num_records,:,:,:] = np.random.uniform(size=(num_records,
                                                     levels.size,
                                                     latitudes.size,
                                                     longitudes.size))

In [ ]:
print(ncfid["forecasts/model"])

In [ ]:
print(ncfid["forecasts/model/temp"])

#### <font color='red'>Close the file</font>

In [ ]:
ncfid.close()

### <font color='red'>Reading a netCDF File</font>

In [ ]:
with Dataset(ncFileName, mode='r') as ncfid:
     time   = ncfid.variables['time'][:]
     lev    = ncfid.variables['lev'][:]
     lat    = ncfid.variables['lat'][:]
     lon    = ncfid.variables['lon'][:]
     temp   = ncfid.variables['temp'][:]
     grpid1 = ncfid.groups['forecasts']
     grpid2 = grpid1.groups['model']
     tempm  = grpid2.variables['temp'][:]                     

In [ ]:
print(lon)

In [ ]:
print(lat)

In [ ]:
print(temp.shape)
print(np.min(temp), np.max(temp))

In [ ]:
print(tempm.shape)
print(np.min(tempm), np.max(tempm))

### <font color='red'>Updating a Variable in an Existing netCDF File</font>

In [ ]:
with Dataset(ncFileName, mode='a') as ncfid:
     temp = ncfid.variables['temp'][:]
     data = temp[:]
     data = 1.1*data + 100.0
     temp[:] = data

In [ ]:
print(temp.shape)
print(np.min(temp), np.max(temp))